## Introducción a CNNs

### Referencias
- LeCun, Y., Bengio, Y. and Hinton, G., 2015. [Deep Learning](https://www.nature.com/articles/nature14539). nature, 521(7553), pp.436-444.
- [Ignite documentation](https://pytorch-ignite.ai/)
- [PyTorch tutorials](https://pytorch.org/tutorials/) 
- [torchmetrics examples](https://torchmetrics.readthedocs.io/en/stable/pages/quickstart.html)
- [visdom](https://github.com/fossasia/visdom)


![pytorch cheatsheet](./figs/pytorch-cheat.jpeg "pytorch cheatsheet")

Figure from [pytorch forum](https://discuss.pytorch.org/t/pytorch-cheat-sheet/72016)

### Que pasos debemos seguir para entrenar una CNN?
- Cargar datos, transformarlos en formato necesario para pytorch. $t_{train}$ va a tener transformaciones diferentes a $t_{test}$, por que? 
- A los datos de entrenamiento debemos dividirlos una vez mas para usar elementos de validación durante el entrenamiento.
- Definir arquitectura de nuestra red (Ver models.py)
- Definir métodos funciones de error, optimizador y métricas de evaluación.

In [43]:
from torchvision import datasets
from torchvision import transforms as tfs
from torch.utils import data
import PIL

transforms_train = [tfs.RandomHorizontalFlip(p=0.7),
                     tfs.RandomAffine(0, scale=(0.7, 1.)),
                     tfs.Resize((64, 64)),
                     tfs.Grayscale(1),
                     tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
                     tfs.ToTensor()
                   ]

transforms_test = [tfs.RandomHorizontalFlip(p=0.7),
                   tfs.RandomAffine(0, scale=(0.7, 1.)),
                   tfs.Resize((64, 64)),
                   tfs.Grayscale(1),
                   tfs.Lambda(lambda x: PIL.ImageOps.invert(x)),
                   tfs.ToTensor()]

train_data = datasets.ImageFolder('../data/train/',
                                     transform=tfs.Compose(transforms_train))

test_data = datasets.ImageFolder('../data/test/',
                                     transform=tfs.Compose(transforms_test))

In [44]:
len(train_data.samples), len(test_data.samples), round(963*0.1)

(963, 319, 96)

In [54]:
splits = data.random_split(train_data, [867, 96])

train_loader = data.DataLoader(splits[0], batch_size=64, shuffle=True)
val_loader = data.DataLoader(splits[1], batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_data, batch_size=64, shuffle=True)

### Ahora tenemos que definir modelo, función de error, optimizador, etc.
- Vamos a ver como usar ignite para entrenamiento supervisado, no se olviden de ver el dashboard the visdom en [http://localhost:8097/](http://localhost:8097/)
- y un simple for loop con torch.


In [50]:
import torch.optim as optim
from torch import nn
from models import _C
import visdom
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
import torch.nn.functional as F
import torch
from aux import create_plot_window
import numpy as np

In [ ]:
C =_C(input_h_w=64)
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    C = C.cuda()
    criterion = criterion.cuda()

C_optimizer = optim.Adam(C.parameters(), lr=0.0003, betas=(0.5, 0.999))
vis = visdom.Visdom()
log_interval = 10

trainer = create_supervised_trainer(C, C_optimizer, F.cross_entropy)
evaluator = create_supervised_evaluator(C, metrics={'accuracy': Accuracy(),
                                                     'ce_ll': Loss(F.cross_entropy)},
                                        device='cpu')

train_avg_loss_window = create_plot_window(vis, '#Iterations', 'Loss', 
                                                 'Training Average Loss')
train_avg_accuracy_window = create_plot_window(vis, '#Iterations', 'Accuracy',
                                                     'Training Average Accuracy')
val_avg_loss_window = create_plot_window(vis, '#Epochs', 'Loss',
                                               'Validation Average Loss')
val_avg_accuracy_window = create_plot_window(vis, '#Epochs', 'Accuracy',
                                                   'Validation Average Accuracy')



@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_nll = metrics['ce_ll']
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(engine.state.epoch, avg_accuracy, avg_nll))
    vis.line(X=np.array([engine.state.epoch]), Y=np.array([avg_accuracy]),
             win=train_avg_accuracy_window, update='append')
    vis.line(X=np.array([engine.state.epoch]), Y=np.array([avg_nll]),
             win=train_avg_loss_window, update='append')

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_nll = metrics['ce_ll']
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(engine.state.epoch, avg_accuracy, avg_nll))
    vis.line(X=np.array([engine.state.epoch]), Y=np.array([avg_accuracy]),
             win=val_avg_accuracy_window, update='append')
    vis.line(X=np.array([engine.state.epoch]), Y=np.array([avg_nll]),
             win=val_avg_loss_window, update='append')

trainer.run(train_loader, max_epochs=100)

Setting up a new session...


Training Results - Epoch: 1  Avg accuracy: 0.58 Avg loss: 2.18
Validation Results - Epoch: 1  Avg accuracy: 0.58 Avg loss: 2.20
Training Results - Epoch: 2  Avg accuracy: 0.61 Avg loss: 2.01
Validation Results - Epoch: 2  Avg accuracy: 0.70 Avg loss: 1.94
Training Results - Epoch: 3  Avg accuracy: 0.73 Avg loss: 1.83
Validation Results - Epoch: 3  Avg accuracy: 0.70 Avg loss: 1.86
Training Results - Epoch: 4  Avg accuracy: 0.75 Avg loss: 1.80
Validation Results - Epoch: 4  Avg accuracy: 0.75 Avg loss: 1.78
Training Results - Epoch: 5  Avg accuracy: 0.76 Avg loss: 1.79
Validation Results - Epoch: 5  Avg accuracy: 0.73 Avg loss: 1.82
Training Results - Epoch: 6  Avg accuracy: 0.76 Avg loss: 1.79
Validation Results - Epoch: 6  Avg accuracy: 0.79 Avg loss: 1.77
Training Results - Epoch: 7  Avg accuracy: 0.80 Avg loss: 1.75
Validation Results - Epoch: 7  Avg accuracy: 0.76 Avg loss: 1.78
Training Results - Epoch: 8  Avg accuracy: 0.82 Avg loss: 1.75
Validation Results - Epoch: 8  Avg accura

### A simple loop here with Pytorch

In [ ]:
# todo

### Evaluation over the test sample

In [ ]:
# todo